In [3]:
import numpy as np
from scipy.stats import norm, multivariate_normal
from scipy.linalg import det, cho_factor

from gproc.generative import sample_at_x
from gproc.kernels import *
from gproc.laplace import laplace_approximation_probit, chol_inverse

In [4]:
N = 500

# x = np.random.uniform(-1, 1, N).reshape(-1, 1) # Reshape to N x 1 matrix
x = np.random.normal(0, 10, N).reshape(-1, 1)

true_lengthscale = 0.1
true_variance = 1.4
kernel = SquaredExponential(lengthscale = true_lengthscale, variance = true_variance)

y, prob_y, f = sample_at_x(x, kernel)

In [6]:
gram = kernel.make_gram(x, x)

# Work out determinant of kernel matrix
inverse_gram, chol_gram = chol_inverse(gram)
log_gram_det = 2 * np.sum(np.log(np.diagonal(chol_gram)))


In [7]:
laplace_mean, df_ll, laplace_cov, objective_history, converged = laplace_approximation_probit(y, inverse_gram)
print(f'Converged: {converged}')

# Get the log determinant and inverse of the approximate covariance matrix
inverse_laplace_cov, chol_laplace_cov = chol_inverse(laplace_cov)

log_laplace_cov_det = 2 * np.sum(np.log(np.diagonal(chol_laplace_cov)))


Converged: True


In [ ]:
def log_joint(y, f, inverse_gram, log_gram_det):
    log_likelihood = np.sum(norm.logcdf(y * f))
    log_prior = -0.5 * ( y.shape[0] * np.log(2 * np.pi) - log_gram_det - f.dot(inverse_gram).dot(f) )
    return log_likelihood + log_prior

log_joint(y, f, inverse_gram, log_gram_det)

-2404.979346362672

In [ ]:
def q_log_pdf(f, inverse_q_cov, log_q_det):
    return -0.5 * ( f.shape[0] * np.log(2 * np.pi) - log_q_det - f.dot(inverse_q_cov).dot(f) )

q_log_pdf(f, inverse_laplace_cov, log_laplace_cov_det)

-1999.9247231061277

In [ ]:
N_imp = 10
f_samples = np.random.multivariate_normal(laplace_mean, laplace_cov, N_imp)

log_joint(y, f_samples[0,:], inverse_gram, log_gram_det)

-2291.9217059256966

In [14]:
def importance_sampler(y, x, q_mean, q_cov, N_imp, gram):
    """
    Implementation of eq 25 that approximates the marginal density p(y|th) on the log scale

    Parameters
    ----------
    iters: float
        number of iterations of the Metropolis Hastings algorithm
        
    y: N dimensional numpy vector
        responses
    
    x: N x D dimensional numpy array
        covariates

    q_mean: N vector
        mean of normal approximation to posterior over latents

    q_cov N x N numpy array
        covariance matrix of normal approximation to posterior over latents
        
    cov: numpy array
        covariance matrix for use in the proposal distribution
        
    N_imp: float
        number of importance samples to use in marginal approximation
    
    gram: N x N numpy array
        kernel gram matrix

    Returns
    ----------
    approximate marginal density, float
    
    """    
    # Inverse of gram matrix and cholesky decomp
    inverse_gram, chol_gram = chol_inverse(gram)
    
    # Work out log-determinant of gram matrix
    log_gram_det = 2 * np.sum(np.log(np.diagonal(chol_gram)))

    # Get the log determinant and inverse of the approximate covariance matrix
    inverse_q_cov, chol_q_cov = chol_inverse(q_cov)
    log_q_det = 2 * np.sum(np.log(np.diagonal(chol_q_cov)))
    
    # Define log joint function
    def log_joint(y, f, inverse_gram, log_gram_det):
        log_likelihood = np.sum(norm.logcdf(y * f))
        log_prior = -0.5 * ( y.shape[0] * np.log(2 * np.pi) + log_gram_det + f.dot(inverse_gram).dot(f) )
        return log_likelihood + log_prior

    # Define log density of Normal approximation q
    def q_log_pdf(f, inverse_q_cov, log_q_det):
        return -0.5 * ( f.shape[0] * np.log(2 * np.pi) + log_q_det + f.dot(inverse_q_cov).dot(f) )

    # Sample latent functions from the normal approximation to full posterior
    f_samples = np.random.multivariate_normal(q_mean, q_cov, N_imp)
    
    # Importance sum
    marg_approx = 0
    for i in range(N_imp):
        
        marg_approx += log_joint(y, f_samples[i, :], inverse_gram, log_gram_det) - q_log_pdf(f_samples[i, :], inverse_q_cov, log_q_det)
    
    return marg_approx/N_imp

In [16]:
importance_sampler(y, x, laplace_mean, laplace_cov, 1000, gram)

-164.1121125826066